# LLM

SEOYEON CHOI  
7/20/23

# Import

In [52]:
import pandas as pd
import torch
from transformers import GPT2Tokenizer, TFGPT2LMHeadModel
import tensorflow as tf

# Data

ref:
[kaggle](https://www.kaggle.com/datasets/mazlumi/mbti-personality-type-twitter-dataset/code)

In [43]:
# df2 = pd.read_csv('./dataset/twitter_MBTI.csv').iloc[:,1:]

In [44]:
df2

In [74]:
def add_quotes(data):
    return f'"{data}"'

In [75]:
df2 = df2.applymap(add_quotes)


In [76]:
df2

In [77]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = TFGPT2LMHeadModel.from_pretrained("gpt2")


All PyTorch model weights were used when initializing TFGPT2LMHeadModel.

All the weights of TFGPT2LMHeadModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.

https://wikidocs.net/152922

https://dacon.io/codeshare/5619

In [78]:
inputs = tokenizer(df2.iloc[:,0],return_tensors="tf", padding=True, truncation=True)